In [1]:
import torch
from torch import nn
import skorch

c:\Users\bhatn\anaconda3\envs\stab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
class MaskedRegression(nn.Module):
    def __init__(self, w):
        super(MaskedRegression, self).__init__()
        self.weighting = nn.Sequential(
            nn.Linear(w, w),
            nn.SELU(),
            nn.Linear(w, w),
            nn.Sigmoid()
        )
        self.regressor = nn.Sequential(
            nn.Linear(w, w),
            nn.SELU()
        )
    def forward(self, X, K):
        X, K = X.float(), K.float()
        X_Known = X * K
        K_weights = self.weighting(K)
        X_pred = self.regressor(X_Known * K_weights)
        return X_pred * (1 - K) + X * K
    def feature_importance(self):
        with torch.no_grad():
            K = torch.randint(0, 2, (1000, self.weighting[0].in_features), dtype=torch.float)
            K_weights = self.weighting(K)
            return torch.sum(K_weights * (1 - K), dim =0)

In [35]:
model = MaskedRegression(22)
model.feature_importance()

tensor([243.4780, 248.5415, 249.3514, 217.1315, 227.4699, 301.4597, 260.5251,
        213.4435, 266.7733, 289.3407, 241.7278, 212.5731, 256.2693, 218.5317,
        274.4241, 265.2117, 246.5060, 224.9903, 299.0406, 286.4236, 268.4144,
        231.8828])

In [13]:
import openml
import pandas as pd
from sklearn.datasets import fetch_openml
import numpy as np

In [14]:
openml.datasets.list_datasets()[197]
fetch_197 = fetch_openml('cpu_act')

c:\Users\bhatn\anaconda3\envs\stab\lib\site-packages\sklearn\datasets\_openml.py:292: UserWarning: Multiple active versions of the dataset matching the name cpu_act exist. Versions may be fundamentally different, returning version 1.
  warn(


In [37]:
data = pd.DataFrame(fetch_197.data)

In [38]:
dataM = data.mask(np.random.random(data.shape) < .1)
missing_mat = torch.from_numpy(dataM.isna().astype(float).values)
M = torch.from_numpy(dataM.fillna(0).astype(float).values)
Truth = torch.from_numpy(data[dataM.isna()].astype(float).values)

In [39]:
# Create dataloader for M
from torch.utils.data import TensorDataset, DataLoader
dataset = TensorDataset(M, missing_mat)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Train model
model = MaskedRegression(M.size(1))

# Define loss function
criterion = nn.MSELoss()

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train model
for epoch in range(100):
    for i, (X, Mb) in enumerate(dataloader):
        optimizer.zero_grad()
        Xhat = model(X, Mb)
        loss = criterion(Xhat * (Mb + 1e-3), X * (Mb + 1e-3))
        loss.backward()
        optimizer.step()


In [40]:
model.eval()

MaskedRegression(
  (weighting): Sequential(
    (0): Linear(in_features=21, out_features=21, bias=True)
    (1): SELU()
    (2): Linear(in_features=21, out_features=21, bias=True)
    (3): Sigmoid()
  )
  (regressor): Sequential(
    (0): Linear(in_features=21, out_features=21, bias=True)
    (1): SELU()
  )
)

In [67]:
aeimpute = model(M, missing_mat) * missing_mat

In [68]:
aeimpute

tensor([[-0.0000, -0.0000, -0.0000,  ..., -0.0000, -1.7581,  0.0000],
        [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
        ...,
        [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -1.7581]],
       dtype=torch.float64, grad_fn=<MulBackward0>)

In [63]:
Truth

tensor([[  nan,   nan,   nan,  ...,   nan,   nan,   nan],
        [  nan,   nan,   nan,  ...,   nan,   nan,   nan],
        [  nan,   77.,   nan,  ...,   nan,   nan,   nan],
        ...,
        [  nan,   nan,   nan,  ...,   nan,   nan,   nan],
        [  nan,    0.,   nan,  ...,   nan, 6355.,   nan],
        [  nan,   nan,   nan,  ...,   nan,   nan,   nan]], dtype=torch.float64)